<a href="https://colab.research.google.com/github/VK-VCS/NLP/blob/main/Next_word_prediction_using_RNN_%2C_LSTM%2C_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re

In [ ]:
def file_to_sentence_list(file_path):
    with open(file_path, 'r') as file:
        text = file.read()

    # Splitting the text into sentences using
    # delimiters like '.', '?', and '!'
    sentences = [sentence.strip() for sentence in re.split(
        r'(?<=[.!?])\s+', text) if sentence.strip()]

    return sentences

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/NLP/Next word Prediction/pizza.txt'
text_data = file_to_sentence_list(file_path)

In [ ]:
text_data

['Pizza, the delectable and iconic dish that has transcended borders and captivated taste buds worldwide, is a testament to the extraordinary fusion of flavors, creativity, and cultural significance.',
 'Originating from the sun-kissed lands of Italy, pizza has evolved into an art form that unites people from diverse backgrounds in a shared love for its mouthwatering combinations.',
 'Its history stretches back centuries, with roots tracing back to ancient civilizations like the Greeks, Romans, and Egyptians, who all had their versions of flatbreads adorned with various ingredients.',
 'However, it was the vibrant city of Naples, Italy, that birthed the pizza we know and adore today.',
 'With its soft and chewy Neapolitan crust, topped with the perfect balance of tomatoes, mozzarella cheese, and fresh basil, the Margherita pizza pays homage to Queen Margherita of Italy and embodies the colors of the Italian flag.',
 'As pizza migrated from the shores of Naples, it found its way to the 

In [ ]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

In [ ]:
total_words

687

In [ ]:
# Create input sequences
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
# Pad sequences
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
    input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
input_sequences

array([[  0,   0,   0, ...,   0,   3,   1],
       [  0,   0,   0, ...,   3,   1, 233],
       [  0,   0,   0, ...,   1, 233,   2],
       ...,
       [  0,   0,   0, ..., 685,   4,  19],
       [  0,   0,   0, ...,   4,  19,  72],
       [  0,   0,   0, ...,  19,  72, 686]], dtype=int32)

In [ ]:
input_sequences[1,:]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,   1,
       233], dtype=int32)

In [ ]:
input_sequences[1, :-1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 1], dtype=int32)

In [ ]:
input_sequences[1,-1]

233

In [ ]:
# split into predictors and label
X, y = input_sequences[:, :-1], input_sequences[:, -1]

In [ ]:
# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [ ]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 10,
                    input_length=max_sequence_len-1))
model.add(SimpleRNN(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model
model.fit(X, y, epochs=500, verbose=1)

Epoch 1/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.0390 - loss: 6.2998
Epoch 2/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.0563 - loss: 5.6151
Epoch 3/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.0600 - loss: 5.7423
Epoch 4/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.0510 - loss: 5.6941
Epoch 5/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.0620 - loss: 5.6681
Epoch 6/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.0655 - loss: 5.6631
Epoch 7/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.0735 - loss: 5.4899
Epoch 8/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.1013 - loss: 5.3138
Epoch 9/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.1229 - loss: 5.0779
Epoch 10/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.1480 - loss: 4.8609
Epoch 11/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.1767 - loss: 4.5751
Epoch 12/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step

In [ ]:
# Generate next word predictions
seed_text = "Pizza have different "
next_words = 5

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Next predicted words: Pizza have different  woven itself into the fabric


In [ ]:
# Define the model
model_LSTM = Sequential()
model_LSTM.add(Embedding(total_words, 10,
                    input_length=max_sequence_len-1))
model_LSTM.add(LSTM(128))
model_LSTM.add(Dense(total_words, activation='softmax'))
model_LSTM.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model_LSTM.fit(X, y, epochs=500, verbose=1)

Epoch 1/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 43ms/step - accuracy: 0.0379 - loss: 6.3738
Epoch 2/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 83ms/step - accuracy: 0.0527 - loss: 5.7201
Epoch 3/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 0.0545 - loss: 5.7074
Epoch 4/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.0601 - loss: 5.6911
Epoch 5/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.0594 - loss: 5.6397
Epoch 6/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.0588 - loss: 5.6105
Epoch 7/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 62ms/step - accuracy: 0.0654 - loss: 5.5338
Epoch 8/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - accuracy: 0.0630 - loss: 5.5133
Epoch 9/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.0729 - loss: 5.4259
Epoch 10/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.0728 - loss: 5.2895
Epoch 11/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.0757 - loss: 5.2603
Epoch 12/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step

In [ ]:
# Generate next word predictions
seed_text = "Pizza have different "
next_words = 5

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model_LSTM.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Next predicted words: Pizza have different  become a cultural icon a


In [ ]:
from tensorflow.keras.layers import GRU

In [ ]:
# Define the model
model_GRU = Sequential()
model_GRU.add(Embedding(total_words, 10,
                    input_length=max_sequence_len-1))
model_GRU.add(GRU(128))
model_GRU.add(Dense(total_words, activation='softmax'))
model_GRU.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model_GRU.fit(X, y, epochs=500, verbose=1)

Epoch 1/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.0378 - loss: 6.4128
Epoch 2/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.0472 - loss: 5.7497
Epoch 3/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.0632 - loss: 5.5774
Epoch 4/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.0628 - loss: 5.6827
Epoch 5/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.0593 - loss: 5.6217
Epoch 6/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - accuracy: 0.0573 - loss: 5.5742
Epoch 7/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.0617 - loss: 5.4468
Epoch 8/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.0656 - loss: 5.3053
Epoch 9/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.0700 - loss: 5.1764
Epoch 10/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.0793 - loss: 5.0179
Epoch 11/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - accuracy: 0.1060 - loss: 4.7769
Epoch 12/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step

In [ ]:
# Generate next word predictions
seed_text = "Pizza have different "
next_words = 5

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model_GRU.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Next predicted words: Pizza have different  become a symbol of comfort
